In [8]:
# import necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt 
from PIL import Image
import torch
import torchvision.transforms as T
import torchvision
import numpy as np 
import cv2
import warnings
warnings.filterwarnings('ignore')
import image_slicer
import os
import tqdm

In [2]:
# load model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# set to evaluation mode
model.eval()

# load the COCO dataset category names
# we will use the same list for this notebook
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

DESIRED_CLASS = [
    'cat',
    'dog',
    'horse',
    'sheep',
    'cow',
    'elephant',
    'bear',
    'zebra',
    'giraffe',
    
]

In [3]:
def get_prediction(img_path, confidence):
  """
  get_prediction
    parameters:
      - img_path - path of the input image
      - confidence - threshold value for prediction score
    method:
      - Image is obtained from the image path
      - the image is converted to image tensor using PyTorch's Transforms
      - image is passed through the model to get the predictions
      - class, box coordinates are obtained, but only prediction score > threshold
        are chosen.
    
  """
  img = Image.open(img_path)
  transform = T.Compose([T.ToTensor()])
  img = transform(img)
  pred = model([img])
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
  pred_score = list(pred[0]['scores'].detach().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x>confidence]
  if(len(pred_t) == 0):
    return [], []
  pred_t = pred_t[-1]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return pred_boxes, pred_class

In [4]:
def detect_object(img_path, confidence=0.5, rect_th=2, text_size=2, text_th=2):
  """
  object_detection_api
    parameters:
      - img_path - path of the input image
      - confidence - threshold value for prediction score
      - rect_th - thickness of bounding box
      - text_size - size of the class label text
      - text_th - thichness of the text
    method:
      - prediction is obtained from get_prediction method
      - for each prediction, bounding box is drawn and text is written 
        with opencv
      - the final image is displayed
  """
  image = cv2.imread(img_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  height, width = image.shape[0:2]
  cow_count = 0
  num_slice = 4
  slices = image_slicer.slice(img_path, num_slice)
  num_slice = len(slices)
  
  
  dim =  int(np.sqrt(num_slice))
  sheight = height // dim
  swidth = width // dim


  image_coordinates = []

  for i in range(0, dim):
    for j in range(0, dim):
      sliced = slices[i * dim + j]
      sliced_path = sliced.generate_filename(prefix=img_path[:-4], path=False)
      boxes, pred_cls = get_prediction(sliced_path, confidence)
      for k in range(len(boxes)):
        if pred_cls[k] in DESIRED_CLASS:
          cow_count += 1
          c1 = (int(boxes[k][0][0] + j * swidth), int(boxes[k][0][1] + i * sheight))
          c2 = (int(boxes[k][1][0] + j * swidth), int(boxes[k][1][1] + i * sheight))
          mid_x = (c2[0] + c1[0]) // 2
          mid_y = (c2[1] + c1[1]) // 2
          image_coordinates.append([mid_x, mid_y])
          cv2.rectangle(image, c1, c2, color=(0, 255, 0), thickness=rect_th)
          cv2.putText(image, 'cow', c1, cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)
      os.remove(sliced_path)
  # plt.figure(figsize=(20, 30))
  # plt.imshow(image)
  return image, cow_count, image_coordinates

In [5]:
img_dir = "cow_images/Fall (cow)/TW4_C1_SW (09012019).JPG"


In [ ]:
# im, cow_count = detect_object(img_dir, confidence=0.6)
# cow_image, cow_count, image_coordinates = detect_object(img_dir, confidence=0.4)
# print("%d number of cows detected" %(cow_count))
# print("Image coordinates: ", image_coordinates)

In [10]:
data = []
f = open("./fasterrcnn_result.txt", 'w')
for folders in tqdm.tqdm(os.listdir("cow_images")):
    for files in tqdm.tqdm(os.listdir("cow_images/" + folders)):
        folder_dir = "cow_images/" + folders + "/"
        file_dir = "cow_images/" + folders + "/" + files
        if os.path.exists(file_dir):
            cow_image, cow_count, image_coordinates = detect_object(file_dir, confidence=0.4)
            new_file_dir = file_dir.replace("cow_images", "cow_images_box")
            cv2.imwrite(new_file_dir, cow_image)
            data.append([new_file_dir, cow_count, image_coordinates])
            f.write("%s, %d" % (new_file_dir, cow_count))
            for pt in image_coordinates:
                f.write(", (%d, %d)" % (pt[0], pt[1]))
            f.write("\n")
f.close()



100%|██████████| 8/8 [2:28:32<00:00, 1114.07s/it]


In [5]:
f.close()

NameError: name 'f' is not defined

In [12]:
print()

In [13]:
# f = open("./fasterrcnn_result.txt", 'w')
# for im in data:
#     f.write("%s, %d" % (im[0], im[1]))
#     for pt in im[2]:
#         f.write(", (%d, %4essee eddddedxd)" % (pt[0], pt[1]))
#     f.write("\n")
# f.close()
        

In [ ]:
i = cv2.imread(img_dir)
cv2.imwrite("cow_images_box/Fall (cow)/TW2_C1_S.JPG", i)

In [ ]:
with open("./result1.csv", 'w') as f:
    f.write("Image Directories, Count\n")
    for result in res:
        res_dir = result[0]
        res_count = result[1]
        res_dir = res_dir[11:]
        print(res_dir, res_count)
        f.write("%s, %d\n" % (res_dir, int(res_count)))